Las clases StructType y StructField de PySpark se utilizan para especificar mediante programación el esquema del DataFrame y crear columnas complejas como struct anidadas, array y map columns. StructType es una colección de objetos StructField que define el nombre de la columna, el tipo de datos de la columna, un booleano para especificar si el campo puede ser anulable o no y metadatos.

Puntos clave:

**Definición de esquemas DataFrame:** StructType se utiliza habitualmente para definir el esquema al crear un DataFrame, sobre todo para datos estructurados con campos de distintos tipos de datos.

**Estructuras anidadas:** Se pueden crear esquemas complejos con estructuras anidadas anidando StructType dentro de otros objetos StructType, lo que permite representar datos jerárquicos o multinivel.

**Aplicación de la estructura de datos:** Al leer datos de diversas fuentes, especificar un StructType como esquema garantiza que los datos se interpretan y estructuran correctamente. Esto es importante cuando se trata de fuentes de datos semiestructuradas o sin esquema.

En este artículo, explicaré diferentes formas de definir la estructura de DataFrame usando StructType con ejemplos de PySpark. Aunque PySpark infiere un esquema a partir de los datos, a veces podemos necesitar definir nuestros propios nombres de columnas y tipos de datos y este artículo explica cómo definir esquemas simples, anidados y complejos.

#1. StructType - Define la estructura del DataFrame

PySpark proporciona la clase StructType de pyspark.sql.types para definir la estructura del DataFrame.
StructType es una colección o lista de objetos StructField.
El método printSchema() de PySpark en el DataFrame muestra las columnas StructType como struct.

#2. StructField - Define los metadatos de la columna DataFrame

PySpark proporciona la clase pyspark.sql.types import StructField para definir las columnas que incluyen nombre de columna(String), tipo de columna (DataType), columna anulable (Boolean) y metadatos (MetaData)

#3. Usando PySpark StructType & StructField con DataFrame

Al crear un PySpark DataFrame podemos especificar la estructura utilizando las clases StructType y StructField. Como se especifica en la introducción, StructType es una colección de StructField que se utiliza para definir el nombre de la columna, tipo de datos, y una bandera para nullable o no. Usando StructField también podemos añadir esquemas struct anidados, ArrayType para arrays, y MapType para pares clave-valor que discutiremos en detalle en secciones posteriores.

El siguiente ejemplo muestra un ejemplo muy simple de cómo crear un StructType & StructField en DataFrame y su uso con datos de ejemplo para soportarlo.

In [0]:
# Imports
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, ArrayType, MapType

spark = SparkSession.builder.master("local[1]") \
                    .appName('SparkByExamples.com') \
                    .getOrCreate()

data = [("James","","Smith","36636","M",3000),
    ("Michael","Rose","","40288","M",4000),
    ("Robert","","Williams","42114","M",4000),
    ("Maria","Anne","Jones","39192","F",4000),
    ("Jen","Mary","Brown","","F",-1)
  ]

schema = StructType([ \
    StructField("firstname",StringType(),True), \
    StructField("middlename",StringType(),True), \
    StructField("lastname",StringType(),True), \
    StructField("id", StringType(), True), \
    StructField("gender", StringType(), True), \
    StructField("salary", IntegerType(), True) \
  ])
 
df = spark.createDataFrame(data=data,schema=schema)
df.printSchema()
df.show(truncate=False)

root
 |-- firstname: string (nullable = true)
 |-- middlename: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: integer (nullable = true)

+---------+----------+--------+-----+------+------+
|firstname|middlename|lastname|id   |gender|salary|
+---------+----------+--------+-----+------+------+
|James    |          |Smith   |36636|M     |3000  |
|Michael  |Rose      |        |40288|M     |4000  |
|Robert   |          |Williams|42114|M     |4000  |
|Maria    |Anne      |Jones   |39192|F     |4000  |
|Jen      |Mary      |Brown   |     |F     |-1    |
+---------+----------+--------+-----+------+------+



#4. Definir estructura anidada de objetos StructType

Cuando trabajamos con DataFrame a menudo necesitamos trabajar con la columna struct anidada y esto se puede definir utilizando StructType.
En el siguiente ejemplo, el tipo de datos de la columna "name" es StructType, que está anidado.

In [0]:
# Defining schema using nested StructType
structureData = [
    (("James","","Smith"),"36636","M",3100),
    (("Michael","Rose",""),"40288","M",4300),
    (("Robert","","Williams"),"42114","M",1400),
    (("Maria","Anne","Jones"),"39192","F",5500),
    (("Jen","Mary","Brown"),"","F",-1)
  ]
structureSchema = StructType([
        StructField('name', StructType([
             StructField('firstname', StringType(), True),
             StructField('middlename', StringType(), True),
             StructField('lastname', StringType(), True)
             ])),
         StructField('id', StringType(), True),
         StructField('gender', StringType(), True),
         StructField('salary', IntegerType(), True)
         ])

df2 = spark.createDataFrame(data=structureData,schema=structureSchema)
df2.printSchema()
df2.show(truncate=False)

root
 |-- name: struct (nullable = true)
 |    |-- firstname: string (nullable = true)
 |    |-- middlename: string (nullable = true)
 |    |-- lastname: string (nullable = true)
 |-- id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: integer (nullable = true)

+--------------------+-----+------+------+
|name                |id   |gender|salary|
+--------------------+-----+------+------+
|{James, , Smith}    |36636|M     |3100  |
|{Michael, Rose, }   |40288|M     |4300  |
|{Robert, , Williams}|42114|M     |1400  |
|{Maria, Anne, Jones}|39192|F     |5500  |
|{Jen, Mary, Brown}  |     |F     |-1    |
+--------------------+-----+------+------+



#5. Añadir y cambiar la estructura del DataFrame

Usando la función SQL de PySpark struct(), podemos cambiar la estructura del DataFrame existente y añadirle un nuevo StructType. El siguiente ejemplo demuestra cómo copiar las columnas de una estructura a otra y añadir una nueva columna. La clase Column de PySpark también proporciona algunas funciones para trabajar con la columna StructType.

In [0]:
# Updating existing structtype using struct
from pyspark.sql.functions import col,struct,when

updatedDF = df2.withColumn("OtherInfo", 
    struct(col("id").alias("identifier"),
    col("gender").alias("gender"),
    col("salary").alias("salary"),
    when(col("salary").cast(IntegerType()) < 2000,"Low")
      .when(col("salary").cast(IntegerType()) < 4000,"Medium")
      .otherwise("High").alias("Salary_Grade")
  )).drop("id","gender","salary")

updatedDF.printSchema()
updatedDF.show(truncate=False)

root
 |-- name: struct (nullable = true)
 |    |-- firstname: string (nullable = true)
 |    |-- middlename: string (nullable = true)
 |    |-- lastname: string (nullable = true)
 |-- OtherInfo: struct (nullable = false)
 |    |-- identifier: string (nullable = true)
 |    |-- gender: string (nullable = true)
 |    |-- salary: integer (nullable = true)
 |    |-- Salary_Grade: string (nullable = false)

+--------------------+------------------------+
|name                |OtherInfo               |
+--------------------+------------------------+
|{James, , Smith}    |{36636, M, 3100, Medium}|
|{Michael, Rose, }   |{40288, M, 4300, High}  |
|{Robert, , Williams}|{42114, M, 1400, Low}   |
|{Maria, Anne, Jones}|{39192, F, 5500, High}  |
|{Jen, Mary, Brown}  |{, F, -1, Low}          |
+--------------------+------------------------+



#6. Uso de SQL ArrayType y MapType

SQL StructType también soporta ArrayType y MapType para definir las columnas DataFrame para las colecciones array y map respectivamente. En el siguiente ejemplo, la columna hobbies se define como ArrayType(StringType) y las propiedades se definen como MapType(StringType,StringType) lo que significa que tanto la clave como el valor son String.

In [0]:
# Using SQL ArrayType and MapType
arrayStructureSchema = StructType([
    StructField('name', StructType([
       StructField('firstname', StringType(), True),
       StructField('middlename', StringType(), True),
       StructField('lastname', StringType(), True)
       ])),
       StructField('hobbies', ArrayType(StringType()), True),
       StructField('properties', MapType(StringType(),StringType()), True)
    ])

#7. Creación de la estructura del objeto StructType a partir de un archivo JSON

Si tienes demasiadas columnas y la estructura del DataFrame cambia de vez en cuando, es una buena práctica cargar el esquema SQL StructType desde un fichero JSON. Puedes obtener el esquema usando df2.schema.json() , almacenarlo en un fichero y usarlo para crear el esquema desde este fichero.

In [0]:
# Using json() to load StructType
print(df2.schema.json())

{"fields":[{"metadata":{},"name":"name","nullable":true,"type":{"fields":[{"metadata":{},"name":"firstname","nullable":true,"type":"string"},{"metadata":{},"name":"middlename","nullable":true,"type":"string"},{"metadata":{},"name":"lastname","nullable":true,"type":"string"}],"type":"struct"}},{"metadata":{},"name":"id","nullable":true,"type":"string"},{"metadata":{},"name":"gender","nullable":true,"type":"string"},{"metadata":{},"name":"salary","nullable":true,"type":"integer"}],"type":"struct"}


{
  "type" : "struct",
  "fields" : [ {
    "name" : "name",
    "type" : {
      "type" : "struct",
      "fields" : [ {
        "name" : "firstname",
        "type" : "string",
        "nullable" : true,
        "metadata" : { }
      }, {
        "name" : "middlename",
        "type" : "string",
        "nullable" : true,
        "metadata" : { }
      }, {
        "name" : "lastname",
        "type" : "string",
        "nullable" : true,
        "metadata" : { }
      } ]
    },
    "nullable" : true,
    "metadata" : { }
  }, {
    "name" : "dob",
    "type" : "string",
    "nullable" : true,
    "metadata" : { }
  }, {
    "name" : "gender",
    "type" : "string",
    "nullable" : true,
    "metadata" : { }
  }, {
    "name" : "salary",
    "type" : "integer",
    "nullable" : true,
    "metadata" : { }
  } ]
}

Alternativamente, también puede utilizar df.schema.simpleString(), esto devolverá un formato de esquema relativamente más simple.

Ahora vamos a cargar el archivo json y utilizarlo para crear un DataFrame.

In [0]:
# Loading json schema to create DataFrame
import json
schemaFromJson = StructType.fromJson(json.loads(schema.json))
df3 = spark.createDataFrame(
        spark.sparkContext.parallelize(structureData),schemaFromJson)
df3.printSchema()

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<command-2522813972922994> in <cell line: 3>()
      1 # Loading json schema to create DataFrame
      2 import json
----> 3 schemaFromJson = StructType.fromJson(json.loads(schema.json))
      4 df3 = spark.createDataFrame(
      5         spark.sparkContext.parallelize(structureData),schemaFromJson)

/usr/lib/python3.9/json/__init__.py in loads(s, cls, object_hook, parse_float, parse_int, parse_constant, object_pairs_hook, **kw)
    337     else:
    338         if not isinstance(s, (bytes, bytearray)):
--> 339             raise TypeError(f'the JSON object must be str, bytes or bytearray, '
    340                             f'not {s.__class__.__name__}')
    341         s = s.decode(detect_encoding(s), 'surrogatepass')

TypeError: the JSON object must be str, bytes or bytearray, not method

#8. Creación de estructura de objeto StructType a partir de cadena DDL

Al igual que cargar una estructura a partir de una cadena JSON, también podemos crearla a partir de una DLL (utilizando la función estática fromDDL() en la clase SQL StructType StructType.fromDDL). También se puede generar DDL a partir de un esquema utilizando toDDL(). printTreeString() en el objeto struct imprime el esquema de forma similar a como lo hace la función printSchema.

In [0]:
from pyspark.sql.types import StructType

# Create StructType from DDL String
ddlSchemaStr = "`fullName` STRUCT<`first`: STRING, `last`: STRING,`middle`: STRING>,`age` INT,`gender` STRING"
ddlSchema = StructType.fromDDL(ddlSchemaStr)
ddlSchema.printTreeString()

---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
<command-2522813972922996> in <cell line: 5>()
      3 # Create StructType from DDL String
      4 ddlSchemaStr = "`fullName` STRUCT<`first`: STRING, `last`: STRING,`middle`: STRING>,`age` INT,`gender` STRING"
----> 5 ddlSchema = StructType.fromDDL(ddlSchemaStr)
      6 ddlSchema.printTreeString()

AttributeError: type object 'StructType' has no attribute 'fromDDL'

#9. Comprobación de la existencia de una columna en un DataFrame

Si queremos realizar algunas comprobaciones sobre los metadatos del DataFrame, por ejemplo, si una columna o campo existe en un DataFrame o el tipo de dato de la columna; podemos hacerlo fácilmente utilizando varias funciones sobre SQL StructType y StructField.

In [0]:
# Checking Column exists using contains()
print(df.schema.fieldNames.contains("firstname"))
print(df.schema.contains(StructField("firstname",StringType,true)))

---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
<command-769984081120396> in <cell line: 2>()
      1 # Checking Column exists using contains()
----> 2 print(df.schema.fieldNames.contains("firstname"))
      3 print(df.schema.contains(StructField("firstname",StringType,true)))

AttributeError: 'function' object has no attribute 'contains'

#10. Ejemplo completo de PySpark StructType & StructField

In [0]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType,ArrayType,MapType
from pyspark.sql.functions import col,struct,when

spark = SparkSession.builder.master("local[1]") \
                    .appName('SparkByExamples.com') \
                    .getOrCreate()

data = [("James","","Smith","36636","M",3000),
    ("Michael","Rose","","40288","M",4000),
    ("Robert","","Williams","42114","M",4000),
    ("Maria","Anne","Jones","39192","F",4000),
    ("Jen","Mary","Brown","","F",-1)
  ]

schema = StructType([ 
    StructField("firstname",StringType(),True), 
    StructField("middlename",StringType(),True), 
    StructField("lastname",StringType(),True), 
    StructField("id", StringType(), True), 
    StructField("gender", StringType(), True), 
    StructField("salary", IntegerType(), True) 
  ])
 
df = spark.createDataFrame(data=data,schema=schema)
df.printSchema()
df.show(truncate=False)

structureData = [
    (("James","","Smith"),"36636","M",3100),
    (("Michael","Rose",""),"40288","M",4300),
    (("Robert","","Williams"),"42114","M",1400),
    (("Maria","Anne","Jones"),"39192","F",5500),
    (("Jen","Mary","Brown"),"","F",-1)
  ]
structureSchema = StructType([
        StructField('name', StructType([
             StructField('firstname', StringType(), True),
             StructField('middlename', StringType(), True),
             StructField('lastname', StringType(), True)
             ])),
         StructField('id', StringType(), True),
         StructField('gender', StringType(), True),
         StructField('salary', IntegerType(), True)
         ])

df2 = spark.createDataFrame(data=structureData,schema=structureSchema)
df2.printSchema()
df2.show(truncate=False)


updatedDF = df2.withColumn("OtherInfo", 
    struct(col("id").alias("identifier"),
    col("gender").alias("gender"),
    col("salary").alias("salary"),
    when(col("salary").cast(IntegerType()) < 2000,"Low")
      .when(col("salary").cast(IntegerType()) < 4000,"Medium")
      .otherwise("High").alias("Salary_Grade")
  )).drop("id","gender","salary")

updatedDF.printSchema()
updatedDF.show(truncate=False)


""" Array & Map"""


arrayStructureSchema = StructType([
    StructField('name', StructType([
       StructField('firstname', StringType(), True),
       StructField('middlename', StringType(), True),
       StructField('lastname', StringType(), True)
       ])),
       StructField('hobbies', ArrayType(StringType()), True),
       StructField('properties', MapType(StringType(),StringType()), True)
    ])

root
 |-- firstname: string (nullable = true)
 |-- middlename: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: integer (nullable = true)

+---------+----------+--------+-----+------+------+
|firstname|middlename|lastname|id   |gender|salary|
+---------+----------+--------+-----+------+------+
|James    |          |Smith   |36636|M     |3000  |
|Michael  |Rose      |        |40288|M     |4000  |
|Robert   |          |Williams|42114|M     |4000  |
|Maria    |Anne      |Jones   |39192|F     |4000  |
|Jen      |Mary      |Brown   |     |F     |-1    |
+---------+----------+--------+-----+------+------+

root
 |-- name: struct (nullable = true)
 |    |-- firstname: string (nullable = true)
 |    |-- middlename: string (nullable = true)
 |    |-- lastname: string (nullable = true)
 |-- id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: integer (nullable = true)

